In [2]:
from six.moves import cPickle
import cv2
import numpy as np
import gym
from gym import wrappers
import sys, os
from functools import partial
from collections import defaultdict
sys.path.append('/home/etoestja/peas')
from peas.networks.rnn import NeuralNetwork
from peas.methods.neat import NEATPopulation, NEATGenotype

In [3]:
class GYMGame(object):
    def __init__(self, game = 'Skiing-v0'):
        self.env = gym.make(game)
        self.actions = self.env.action_space.n
    # Resizing to black-white 42x42
    def resize_frame(self, frame):
        frame = frame[34:34+160, :160]
        # Resize by half, then down to 42x42 (essentially mipmapping). If
        frame = cv2.resize(frame, (84, 84))
        frame = cv2.resize(frame, (42, 42))
        frame = cv2.resize(frame, (10, 10))
        frame = frame.mean(2)
        frame = frame.astype(np.float32)
        frame *= (1.0 / 255.0)
        return frame
    # a = argmax_a Q(s,a)
    def predict_action(self, observation, network):
        #return env.action_space.sample()
        
        #compressed_observation = self.resize_frame(observation).flatten()
        
        # Dirty hack with console's RAM
        compressed_observation = self.env._get_ram()/255.*2-1
        s = network.feed(compressed_observation)[-self.actions:]
        action = np.argmax(s)
        return(action)
    def solve(self, network):
        return False
    def evaluate(self, network):
        if not isinstance(network, NeuralNetwork):
            network = NeuralNetwork(network)
        
        observation = self.env.reset()
        done = False
        iteration, total_reward = 0, 0
        
        while not done:
            #env.render()
            action = self.predict_action(observation, network)
            observation, reward, done, info = self.env.step(action)
            total_reward += reward

            #if iteration % 500 == 0:
                #print(str(iteration))

            if total_reward < -12000 or iteration >= 8000:
                break

            iteration += 1

        
        res = {'fitness': total_reward, 'steps': iteration}
        #print res
        return res
    def genotype(self):
        return lambda: NEATGenotype(inputs=128,
                                outputs = self.actions,
                                weight_range=(-3,3),
                                types=['tanh'])

In [4]:
# Create game
game = GYMGame()

# Genotype of the network
genotype = game.genotype()

[2017-02-06 15:00:57,179] Making new env: Skiing-v0


In [ ]:
# Create a population
pop = NEATPopulation(genotype, popsize=50, max_cores=10)

# Run the evolution, tell it to use the task as an evaluator
pop.epoch(generations=100, evaluator=game, solution=game)

Running in 10 processes.


# Adjusting image

In [ ]:
observation, reward, done, info = game.env.step(0)

In [26]:
from scipy.misc import imsave

In [33]:
frame = observation[50:50+160, 30:-30]
frame = cv2.resize(frame, (84, 84))
frame = cv2.resize(frame, (30, 30))
frame = frame.mean(2)
frame = frame.astype(np.float32)
frame *= (1.0 / 255.0)

In [34]:
imsave('outfile.png', frame)

# Discovering ram

In [55]:
dir(game.env)

['__class__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__doc__',
 '__format__',
 '__getattribute__',
 '__getstate__',
 '__hash__',
 '__init__',
 '__module__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_action_set',
 '_buffer',
 '_close',
 '_closed',
 '_configure',
 '_configured',
 '_env_closer_id',
 '_ezpickle_args',
 '_ezpickle_kwargs',
 '_get_image',
 '_get_obs',
 '_get_ram',
 '_n_actions',
 '_obs_type',
 '_owns_render',
 '_render',
 '_reset',
 '_seed',
 '_step',
 '_unwrapped',
 'action_space',
 'ale',
 'close',
 'configure',
 'frameskip',
 'game_path',
 'get_action_meanings',
 'metadata',
 'monitor',
 'np_random',
 'observation_space',
 'render',
 'reset',
 'reward_range',
 'seed',
 'spec',
 'step',
 'unwrapped',
 'viewer']

In [12]:
game.env._get_ram()/255.*2-1

array([-0.38039216, -0.89803922,  0.9372549 , -0.88235294,  0.9372549 ,
       -0.90588235,  0.9372549 , -0.89019608,  0.9372549 , -0.92156863,
        0.9372549 , -1.        , -1.        , -1.        ,  0.00392157,
       -0.9372549 , -0.94509804, -1.        ,  0.00392157, -0.49803922,
        0.00392157,  0.83529412,  0.92941176, -0.01176471,  0.9372549 ,
       -0.40392157, -0.05882353, -1.        , -0.99215686,  0.05098039,
       -0.78039216,  0.83529412, -0.37254902, -0.95294118, -0.59215686,
       -0.23137255,  0.25490196,  0.74117647,  0.00392157, -1.        ,
       -0.98431373,  0.04313725,  0.04313725, -0.96862745, -0.98431373,
        0.02745098, -0.01176471,  0.23137255, -0.12941176, -0.01176471,
       -0.01176471,  0.23137255, -0.12941176, -0.01176471,  1.        ,
        1.        ,  1.        ,  1.        , -0.75686275, -0.94509804,
       -0.88235294, -0.75686275, -0.38039216, -0.94509804, -0.76470588,
        0.21568627,  0.19215686, -0.5372549 , -0.67843137,  0.00